<a href="https://colab.research.google.com/github/DonghaeSuh/NLP_Pytorch/blob/main/Model/GPT_2/GPT2_GEN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sentencepiece : https://github.com/google/sentencepiece/blob/master/python/README.md

### NSMC

In [21]:
import urllib

In [22]:
urllib.request.urlretrieve('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt','train.txt')

('train.txt', <http.client.HTTPMessage at 0x7fdd6c8720e0>)

In [23]:
temp=[]

In [24]:
with open('train.txt','r') as f:
  f.readline()
  for sent in f.readlines():
    data = sent.split('\t')
    temp.append(data[1])

In [25]:
cd drive/MyDrive/Pytorch\ NLP/GPT-2/data_in/gpt2_ckpt

/content/drive/MyDrive/Pytorch NLP/GPT-2/data_in/gpt2_ckpt


In [26]:
with open('corpus.txt','w') as f:
  for sent in temp:
    f.write(f'\n {sent}')

### Sentencepiece

https://keep-steady.tistory.com/7

In [12]:
pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.0 MB/s eta 0:00:00


In [13]:
import sentencepiece as spm

In [27]:
 spm.SentencePieceTrainer.train(input='corpus.txt',model_prefix='kor_tokenizer',vocab_size=50000, model_type='bpe',max_sentence_length=9999999,pad_id=0,unk_id=1,bos_id=2,eos_id=3)

### gluonnlp

In [28]:
pip install gluonnlp==0.9.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.9.1-cp310-cp310-linux_x86_64.whl size=530292 sha256=9acf0f74a0ee0f964e24c3cf3f1a7f964fd64473ef97a4506e09490cc781eafb
  Stored in directory: /root/.cache/pip/wheels/fc/5b/9c/3295bb07f7c5544a96303a48988707816f44a536e8e1413922
Successfully built gluonnlp


In [29]:
pip install mxnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 7.6 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1


In [30]:
import gluonnlp as nlp

In [31]:
from gluonnlp.data import SentencepieceTokenizer

In [33]:
tokenizer = SentencepieceTokenizer('kor_tokenizer.model')

In [35]:
sp=spm.SentencePieceProcessor()
sp.Load('kor_tokenizer.model')

True

In [54]:
sp.EncodeAsPieces('안녕하세요 저는 사람이에요.')

['▁안녕하세요', '▁저는', '▁사람이', '에요', '.']

In [55]:
sp.encode('안녕하세요 저는 사람이에요.')

[20363, 1619, 1047, 802, 48289]

In [38]:
vocab = nlp.vocab.BERTVocab.from_sentencepiece('kor_tokenizer.model')
# gluonnlp에서 sentnecnepice tokenizer에서 vocab을 직접적으로 가져올 수 있는 방법이 이 방법뿐
# => BERT가 아님에도 BERT 전용 Vocab을 사용. => 사용하는데 상관이 없기는 함.

In [48]:
print('<pad> :{}, <unk> :{} , <s> :{}, </s> :{} '.format(vocab['<pad>'],vocab['<unk>'],vocab['<s>'],vocab['</s>']))

<pad> :0, <unk> :1 , <s> :2, </s> :3 


### config & model_weight

In [56]:
urllib.request.urlretrieve('https://huggingface.co/gpt2/resolve/main/config.json','config.json')

('config.json', <http.client.HTTPMessage at 0x7fdd07c1ada0>)

In [57]:
urllib.request.urlretrieve('https://huggingface.co/gpt2/resolve/main/model.safetensors','model.safetensors')

('model.safetensors', <http.client.HTTPMessage at 0x7fdd07c19ed0>)

## Top k & Top p

1